In [2]:
# Cell 1: imports & basic config

import json
from dataclasses import dataclass, asdict
from typing import List, Optional, Dict, Any
from dotenv import load_dotenv
import os

import numpy as np

# If you're using the OpenAI client:
from openai import OpenAI

# load_dotenv()  # looks for .env in current dir
client_oa = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))  # or use env var

# Embedding model name (adjust to what you're actually using)
EMBEDDING_MODEL = "text-embedding-3-small"

# Chat model for intent parsing
INTENT_MODEL = "gpt-4.1-mini"  # or whatever you're on


In [3]:
# Cell 2: load YANG catalog

CATALOG_PATH = "../data/sensor_catalog.jsonl"  # or .json

all_rows = []

with open(CATALOG_PATH, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue
        all_rows.append(json.loads(line))

len(all_rows)


54063

In [4]:
# Cell 3: define intent schema

@dataclass
class TelemetryIntent:
    # High-level task
    task: str                      # e.g. "generate_telemetry_config"

    # Protocol / domain
    protocol_tag: Optional[str]    # e.g. "bgp", "ospf", "qos"
    vendor: Optional[str]          # e.g. "cisco"
    os: Optional[str]              # e.g. "ios xr"

    # Transport / export details
    transport: Optional[str]       # e.g. "grpc"
    tls: Optional[bool]            # True / False / None
    destination_address: Optional[str]
    destination_port: Optional[int]

    # Telemetry sampling / style
    sample_interval_ms: Optional[int]  # if user mentions "every 30 seconds"

    # What kind of data they care about
    metric_focus: List[str]       # e.g. ["neighbors", "stats", "status"]
    extra_requirements: str       # free-text notes (e.g. "no tls", "self-describing-gpb")

    # Raw original query (so you can always fall back)
    original_query: str

    # Helper: construct from dict safely
    @staticmethod
    def from_dict(d: Dict[str, Any]) -> "TelemetryIntent":
        return TelemetryIntent(
            task=d.get("task", "generate_telemetry_config"),
            protocol_tag=d.get("protocol_tag"),
            vendor=d.get("vendor"),
            os=d.get("os"),
            transport=d.get("transport"),
            tls=d.get("tls"),
            destination_address=d.get("destination_address"),
            destination_port=d.get("destination_port"),
            sample_interval_ms=d.get("sample_interval_ms"),
            metric_focus=d.get("metric_focus") or [],
            extra_requirements=d.get("extra_requirements", ""),
            original_query=d.get("original_query", ""),
        )


In [5]:
# Cell 4: build prompt for the intent LLM

INTENT_SYSTEM_PROMPT = """
You are a network telemetry assistant. Your job is to convert a natural language request
into a structured JSON object describing what telemetry configuration is needed.

Follow these rules:
- Only output valid JSON, no comments, no explanations.
- Use the following keys exactly:

{
  "task": string,                     // e.g. "generate_telemetry_config"
  "protocol_tag": string or null,     // one of: bgp, ospf, isis, mpls, ldp, multicast, routing, interfaces, qos, acl, platform, tunnel, or null
  "vendor": string or null,           // e.g. "cisco"
  "os": string or null,               // e.g. "ios xr"
  "transport": string or null,        // e.g. "grpc"
  "tls": boolean or null,             // true, false, or null if not specified
  "destination_address": string or null,
  "destination_port": integer or null,
  "sample_interval_ms": integer or null,   // telemetry sampling interval in ms if mentioned
  "metric_focus": array of strings,   // short tags like "neighbors", "routes", "stats", "summary", "status"
  "extra_requirements": string,       // any additional constraints (formats, comments, etc.)
  "original_query": string            // echo the original user query
}

If the user doesn't specify something, set it to null (or [] for arrays) instead of guessing too much.
When inferring protocol_tag, map phrases like "border gateway protocol" to "bgp".
""".strip()


In [6]:
# Cell 5: LLM call -> JSON -> TelemetryIntent

def parse_intent(query: str) -> TelemetryIntent:
    response = client_oa.chat.completions.create(
        model=INTENT_MODEL,
        messages=[
            {"role": "system", "content": INTENT_SYSTEM_PROMPT},
            {"role": "user", "content": query},
        ],
        temperature=0.0,
    )

    content = response.choices[0].message.content
    # content should be pure JSON per instructions
    try:    # Ensure original_query is filled

        data = json.loads(content)
    except json.JSONDecodeError as e:
        raise ValueError(f"Model did not return valid JSON: {e}\n\n{content}")

    # Ensure original_query is filled
    if not data.get("original_query"):
        data["original_query"] = query

    return TelemetryIntent.from_dict(data)
    # return data


In [7]:
# Cell 6: test with your sample query

query = (
    "generate telemetry configuration for cisco ios xr about bgp protocol ? "
    "Use grpc with no tls, the telemetry server address is 192.0.2.0 with port 57500. "
    "Choose relevant sensor-paths."
)

intent = parse_intent(query)
intent


TelemetryIntent(task='generate_telemetry_config', protocol_tag='bgp', vendor='cisco', os='ios xr', transport='grpc', tls=False, destination_address='192.0.2.0', destination_port=57500, sample_interval_ms=None, metric_focus=['neighbors', 'routes', 'summary', 'status'], extra_requirements='Use relevant BGP sensor-paths for telemetry in Cisco IOS XR.', original_query='generate telemetry configuration for cisco ios xr about bgp protocol ? Use grpc with no tls, the telemetry server address is 192.0.2.0 with port 57500. Choose relevant sensor-paths.')

In [8]:
# Canonical: TelemetryIntent -> text for embeddings

def intent_to_search_text(intent: TelemetryIntent) -> str:
    """
    Convert a TelemetryIntent into a rich, structured text representation
    suitable for semantic embedding and vector search over YANG sensor paths.
    """

    parts: list[str] = []

    # High-level summary first (good for embeddings)
    summary_bits = []
    if intent.protocol_tag:
        summary_bits.append(f"{intent.protocol_tag.upper()}")
    if intent.vendor:
        summary_bits.append(intent.vendor)
    if intent.os:
        summary_bits.append(intent.os)
    proto_stack = " ".join(summary_bits) if summary_bits else "unspecified protocol"

    parts.append(f"Telemetry intent: generate telemetry configuration for {proto_stack}")

    # Core fields
    parts.append(f"Task: {intent.task}")

    if intent.protocol_tag:
        parts.append(f"Protocol: {intent.protocol_tag}")

    if intent.vendor:
        parts.append(f"Vendor: {intent.vendor}")

    if intent.os:
        parts.append(f"OS: {intent.os}")

    # Transport + security
    if intent.transport or intent.tls is not None:
        if intent.tls is False:
            security_str = "no-tls"
        elif intent.tls is True:
            security_str = "tls"
        else:
            security_str = "unspecified"
        if intent.transport:
            parts.append(f"Transport: {intent.transport} ({security_str})")
        else:
            parts.append(f"Security: {security_str}")

    # Destination (collector) info
    if intent.destination_address or intent.destination_port:
        addr = intent.destination_address or "unspecified-address"
        port = intent.destination_port if intent.destination_port is not None else "unspecified-port"
        parts.append(f"Destination: {addr}:{port}")

    # Sampling interval
    if intent.sample_interval_ms:
        parts.append(f"Sample interval (ms): {intent.sample_interval_ms}")

    # What metrics we care about (matches your YANG categories nicely)
    if intent.metric_focus:
        parts.append("Metric focus: " + ", ".join(intent.metric_focus))

    # Extra requirements from the planner / user
    if intent.extra_requirements:
        parts.append("Extra requirements: " + intent.extra_requirements)

    # Always include the original query at the end; it's pure semantic gold
    if intent.original_query:
        parts.append("Original query: " + intent.original_query)

    return "\n".join(parts)


In [9]:
intent_text = intent_to_search_text(intent)
intent_text

'Telemetry intent: generate telemetry configuration for BGP cisco ios xr\nTask: generate_telemetry_config\nProtocol: bgp\nVendor: cisco\nOS: ios xr\nTransport: grpc (no-tls)\nDestination: 192.0.2.0:57500\nMetric focus: neighbors, routes, summary, status\nExtra requirements: Use relevant BGP sensor-paths for telemetry in Cisco IOS XR.\nOriginal query: generate telemetry configuration for cisco ios xr about bgp protocol ? Use grpc with no tls, the telemetry server address is 192.0.2.0 with port 57500. Choose relevant sensor-paths.'

In [10]:
from openai import OpenAI
from qdrant_client.models import PointStruct

EMBEDDING_MODEL = "text-embedding-3-small"

def get_embedding(text: str) -> list[float]:
    resp = client_oa.embeddings.create(
        model=EMBEDDING_MODEL,
        input=text,
    )
    return resp.data[0].embedding

query_vector = get_embedding(intent_text)


In [11]:
from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance

client = QdrantClient(host="localhost", port=6333)

points, next_page = client.scroll(
    collection_name="catalog_embeddings",
    with_payload=True,
    with_vectors=False,
    limit=5,
)

for p in points[:3]:
    print("id:", p.id)
    print("payload:", p.payload)
    print("protocol_tag in payload:", p.payload.get("protocol_tag"))
    print("---")


id: 0
payload: {'yang_id': 0, 'module': 'Cisco-IOS-XR-tunnel-ip-ma-oper', 'path': 'Cisco-IOS-XR-tunnel-ip-ma-oper:tunnel-ip-ma', 'protocol_tag': 'tunnel', 'category': ['state', 'stats', 'tunnels'], 'kind': 'container', 'leaf_count': 92, 'description': 'Tunnel Ip ma parameters', 'leaf_names': ['address-family', 'adjacency', 'afi', 'bandwidth', 'base-caps-state', 'bfd-session-state', 'cap-ipv4-transport-supported', 'cap-ipv6-transport-supported', 'check-point-id', 'convergence-state', 'destination-address-length', 'destination-prefix-list', 'endpt-count', 'endpt-prod', 'eod-recvd', 'ep-app-id', 'flag-bits', 'flags', 'flags-other', 'gre-cap-checksum-supported', 'gre-cap-ipv4-transport-supported', 'gre-cap-ipv6-transport-supported', 'gre-cap-key-supported', 'gre-cap-max-mtu-supported', 'gre-cap-max-tunnels-supported', 'gre-cap-mgre-ipv4-transport-supported', 'gre-cap-mgre-ipv6-transport-supported', 'gre-cap-multi-encap-supported', 'gre-cap-platform-supported', 'gre-cap-seq-num-supported', 

In [ ]:
from qdrant_client.models import Filter, FieldCondition, MatchValue  # ← Your existing filter imports (unchanged)

# Your filter stays exactly the same
query_filter = Filter(
    must=[
        FieldCondition(
            key="protocol_tag",
            match=MatchValue(value=intent.protocol_tag)  # ← Assumes intent.protocol_tag is e.g., a string/int
        )
    ]
)

# First search (with filter) — no Query/QueryVector wrapper needed for simple vector queries
hits = client.query_points(
    collection_name="catalog_embeddings",
    query=query_vector,                          # ← Pass vector directly (list[float])
    query_filter=query_filter,                   # ← Your protocol_tag filter
    limit=10,
    with_payload=True,                        # ← Equivalent to with_payload=True (default)
).points  # ← Extract the list of points (same as old hits)

# No matches with protocol filter? Retry without it.
if not hits and query_filter is not None:
    hits = client.query_points(
        collection_name="catalog_embeddings",
        query=query_vector,                          # ← Still direct
        limit=10,
        with_payload=True,
    ).points

# Your print loop unchanged!
for h in hits:
    print(h.id, h.payload["yang_id"], h.payload["path"])

5 5 Cisco-IOS-XR-tunnel-ip-ma-oper:tunnel-ip-ma/database/transport-vrf-datas/transport-vrf-data
4 4 Cisco-IOS-XR-tunnel-ip-ma-oper:tunnel-ip-ma/database/transport-vrf-datas
0 0 Cisco-IOS-XR-tunnel-ip-ma-oper:tunnel-ip-ma
3 3 Cisco-IOS-XR-tunnel-ip-ma-oper:tunnel-ip-ma/database
1 1 Cisco-IOS-XR-tunnel-ip-ma-oper:tunnel-ip-ma/gsp-node-db-summary
2 2 Cisco-IOS-XR-tunnel-ip-ma-oper:tunnel-ip-ma/gsp-node-db-summary/gspdb-array
6 6 Cisco-IOS-XR-tunnel-ip-ma-oper:tunnel-ip-ma/database/transport-vrf-datas/transport-vrf-data/idb-array
7 7 Cisco-IOS-XR-tunnel-ip-ma-oper:tunnel-ip-ma/database/transport-vrf-datas/transport-vrf-data/idb-array/source-address
9 9 Cisco-IOS-XR-tunnel-ip-ma-oper:tunnel-ip-ma/database/tunnel-ids
8 8 Cisco-IOS-XR-tunnel-ip-ma-oper:tunnel-ip-ma/database/transport-vrf-datas/transport-vrf-data/idb-array/destination-address


In [29]:
# --- Cell X: config-plan schema for Jinja rendering ---

from dataclasses import dataclass
from typing import List, Dict, Any, Optional

@dataclass
class TelemetryDestination:
    address: str
    port: int
    encoding: str          # e.g. "self-describing-gpb"
    protocol: str          # e.g. "grpc"
    tls: bool              # False => "no-tls"

@dataclass
class TelemetryDestinationGroup:
    name: str              # e.g. "DG-GRPC"
    address_family: str    # "ipv4" or "ipv6"
    destinations: List[TelemetryDestination]

@dataclass
class TelemetrySensorGroup:
    name: str              # e.g. "BGP-STATS"
    sensor_paths: List[str]

@dataclass
class TelemetrySubscription:
    name: str              # e.g. "SUB-BGP"
    sensor_group: str      # must match TelemetrySensorGroup.name
    sample_interval_ms: int
    destination_group: str # must match TelemetryDestinationGroup.name
    mode: str              # "sample" or "on-change"

@dataclass
class TelemetryConfigPlan:
    sensor_groups: List[TelemetrySensorGroup]
    subscriptions: List[TelemetrySubscription]
    destination_groups: List[TelemetryDestinationGroup]
    notes: str = ""

    @staticmethod
    def from_dict(d: Dict[str, Any]) -> "TelemetryConfigPlan":
        sg_objs = [
            TelemetrySensorGroup(
                name=sg["name"],
                sensor_paths=sg.get("sensor_paths", []),
            )
            for sg in d.get("sensor_groups", [])
        ]

        sub_objs = [
            TelemetrySubscription(
                name=sub["name"],
                sensor_group=sub["sensor_group"],
                sample_interval_ms=sub["sample_interval_ms"],
                destination_group=sub["destination_group"],
                mode=sub.get("mode", "sample"),
            )
            for sub in d.get("subscriptions", [])
        ]

        dg_objs = []
        for dg in d.get("destination_groups", []):
            dest_objs = [
                TelemetryDestination(
                    address=dest["address"],
                    port=dest["port"],
                    encoding=dest["encoding"],
                    protocol=dest["protocol"],
                    tls=dest["tls"],
                )
                for dest in dg.get("destinations", [])
            ]
            dg_objs.append(
                TelemetryDestinationGroup(
                    name=dg["name"],
                    address_family=dg["address_family"],
                    destinations=dest_objs,
                )
            )

        return TelemetryConfigPlan(
            sensor_groups=sg_objs,
            subscriptions=sub_objs,
            destination_groups=dg_objs,
            notes=d.get("notes", ""),
        )


In [30]:
# --- Cell Y: system prompt + model for config planner ---

CONFIG_MODEL = "gpt-4.1-mini"  # reuse mini, or bump if needed

CONFIG_PLANNER_SYSTEM_PROMPT = """
You are a Cisco IOS XR telemetry configuration planner.

Given:
- the original natural-language query,
- a normalized TelemetryIntent object,
- and a small list of candidate YANG sensor paths,

you must output a single JSON object that DESCRIBES the configuration (a plan),
not the CLI itself.

This JSON will feed a Jinja2 template that renders `telemetry model-driven` CLI.

GENERAL RULES
- Output ONLY valid JSON. No comments, no markdown, no explanations.
- Use ONLY sensor paths that appear in `candidate_sensor_paths`. Never invent new YANG paths.
- Prefer the highest-scoring, most specific operational BGP paths if protocol_tag == "bgp".
- When possible, use values from the TelemetryIntent (transport, tls, address, port, sample_interval_ms).

SCHEMA (you MUST respect this):

{
  "sensor_groups": [
    {
      "name": string,                 // e.g. "BGP-STATS"
      "sensor_paths": [ string, ... ] // subset of candidate paths
    }
  ],
  "subscriptions": [
    {
      "name": string,                 // e.g. "SUB-BGP"
      "sensor_group": string,         // must match some sensor_groups[i].name
      "sample_interval_ms": integer,  // from intent.sample_interval_ms, else default 30000
      "destination_group": string,    // must match some destination_groups[i].name
      "mode": string                  // "sample" or "on-change"; default "sample"
    }
  ],
  "destination_groups": [
    {
      "name": string,                 // e.g. "DG-GRPC"
      "address_family": string,       // "ipv4" or "ipv6"
      "destinations": [
        {
          "address": string,          // from intent.destination_address if available
          "port": integer,            // from intent.destination_port if available
          "encoding": string,         // e.g. "self-describing-gpb"
          "protocol": string,         // e.g. "grpc"
          "tls": boolean              // true => TLS, false => "no-tls"
        }
      ]
    }
  ],
  "notes": string                     // optional free-text notes; "" if nothing special
}

NAMING CONVENTIONS (unless user explicitly says otherwise):
- If intent.protocol_tag is available, let P = intent.protocol_tag.upper().
  * sensor-group name:    P + "-STATS"   (e.g. "BGP-STATS")
  * subscription name:    "SUB-" + P     (e.g. "SUB-BGP")
- If intent.transport is available, let T = intent.transport.upper().
  * destination-group name: "DG-" + T   (e.g. "DG-GRPC")

MAPPING FROM INTENT:
- protocol_tag: helps you pick which sensor paths to use.
- transport: typically "grpc" -> set destination.protocol = "grpc".
- tls:
    * true  -> TLS enabled
    * false -> "no-tls" in CLI, so set json field "tls": false
- destination_address and destination_port MUST be used when present.
- sample_interval_ms:
    * use if set
    * else default to 30000 (30 seconds)
- metric_focus: can guide which candidate paths you keep (neighbors, routes, stats, etc).
- If destination_address looks like IPv4, set "address_family": "ipv4". If IPv6, "ipv6".

If something is truly unspecified, pick simple, reasonable defaults but keep them consistent and minimal.
Do NOT invent strange encodings; for gRPC use "self-describing-gpb" unless the user explicitly forces another encoding.
""".strip()


In [32]:
# --- Cell Z: LLM function to build the structured config JSON ---

from dataclasses import asdict

def plan_telemetry_config(
    query: str,
    intent: TelemetryIntent,
    hits: List[Any],
    model: str = CONFIG_MODEL,
) -> TelemetryConfigPlan:
    """
    Use a second LLM to turn:
      - the original query,
      - a TelemetryIntent,
      - and a list of Qdrant hits (sensor paths)
    into a TelemetryConfigPlan that can be rendered by a Jinja template.
    """

    # 1) Flatten Qdrant hits into a small JSON-friendly structure
    candidate_sensor_paths: List[Dict[str, Any]] = []
    for h in hits:
        payload = getattr(h, "payload", None) or {}
        path = payload.get("path")
        if not path:
            continue
        candidate_sensor_paths.append(
            {
                "path": path,
                "yang_id": payload.get("yang_id"),
                "description": payload.get("description"),
                "protocol_tag": payload.get("protocol_tag"),
                "score": getattr(h, "score", None),
            }
        )

    # Safety: if somehow empty, it's still better to send an empty list
    # and let the LLM possibly set sensor_paths = [] than to hallucinate
    # paths on our side.

    # 2) Bundle everything as a single JSON payload for the LLM
    planner_input = {
        "original_query": query,
        "intent": asdict(intent),
        "candidate_sensor_paths": candidate_sensor_paths,
    }

    response = client_oa.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": CONFIG_PLANNER_SYSTEM_PROMPT},
            {"role": "user", "content": json.dumps(planner_input)},
        ],
        temperature=0.0,
    )

    content = response.choices[0].message.content

    # 3) Parse JSON returned by model
    try:
        data = json.loads(content)
    except json.JSONDecodeError as e:
        raise ValueError(f"Config planner model did not return valid JSON: {e}\n\n{content}")

    # 4) Convert dict -> TelemetryConfigPlan dataclass
    return TelemetryConfigPlan.from_dict(data)


In [33]:
# You already have:
# query (str)
# intent = parse_intent(query)
# hits = client.query_points(...).points

config_plan = plan_telemetry_config(query, intent, hits)

# If you want to inspect the JSON:
print(json.dumps(asdict(config_plan), indent=2))


{
  "sensor_groups": [
    {
      "name": "BGP-STATS",
      "sensor_paths": []
    }
  ],
  "subscriptions": [
    {
      "name": "SUB-BGP",
      "sensor_group": "BGP-STATS",
      "sample_interval_ms": 30000,
      "destination_group": "DG-GRPC",
      "mode": "sample"
    }
  ],
  "destination_groups": [
    {
      "name": "DG-GRPC",
      "address_family": "ipv4",
      "destinations": [
        {
          "address": "192.0.2.0",
          "port": 57500,
          "encoding": "self-describing-gpb",
          "protocol": "grpc",
          "tls": false
        }
      ]
    }
  ],
  "notes": "No BGP sensor paths available in candidate_sensor_paths; empty sensor group created."
}


In [37]:
from jinja2 import Environment, FileSystemLoader

env = Environment(loader=FileSystemLoader("../data/templates"))
template = env.get_template("telemetry_ios_xr.j2")

rendered_cli = template.render(**asdict(config_plan))
print(rendered_cli)


telemetry model-driven


 sensor-group BGP-STATS

 !




 subscription SUB-BGP

  sensor-group-id BGP-STATS sample-interval 30000  ! every 30 seconds

  destination-group DG-GRPC
 !




 destination-group DG-GRPC
  address-family ipv4

   destination 192.0.2.0
    port 57500
    encoding self-describing-gpb
    protocol grpc no-tls
   !

  !

!


! NOTES:
! No BGP sensor paths available in candidate_sensor_paths; empty sensor group created.

commit
